In [1]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

In [ ]:
(df[pd.isna(df["region_1"]) & pd.isna(df["region_2"])])

In [ ]:
df.info()

In [ ]:
df

In [ ]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(df["designation"].value_counts())

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df["designation"].value_counts()[:100])

In [ ]:
#df[df["designation_tr"] == "reserva" | df["designation_tr"] == "riserva" | df["designation_tr"] == "rèserve"]["designation_tr"] = "reserve"
df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","rèserve"],["reserve","reserve","reserve"])


In [ ]:
df

In [ ]:
# rimuovo regioni doppie e creo regione unica
df.loc[df["region_1"] == df["region_2"], "region_2"] = ""
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)

In [ ]:
singular_region_complete = set(df["region_complete"])
len(singular_region_complete)

In [ ]:
import geocoder

g = geocoder.arcgis('Redlands, CA')
print(g.latlng)

In [ ]:
import geocoder

singular_region_complete_with_latlong = []
for region in singular_region_complete:
    print(region)
    loc =  geocoder.arcgis(region,maxRows=1)
    print(loc)
    try:
        singular_region_complete_with_latlong.append((region, loc.latlng[0],loc.latlng[1]))   
    except:
        pass

In [ ]:
with open('region_coordinates.csv', 'w') as fp:
    fp.write('\n'.join('%s,%f,%f' % x for x in singular_region_complete_with_latlong))

In [ ]:
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df = pd.merge(df, df_region_coordinates, how="left", on="region_complete")

In [ ]:
df.to_csv("competition_dataset/mine_dev.csv",index=False)

In [ ]:
df = pd.get_dummies(df, columns=["designation_tr","province","variety"])

In [ ]:
df.columns

In [10]:
df = pd.read_csv("competition_dataset/mine_dev.csv")

In [61]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    df["designation_tr"].value_counts()[:200]
    
len(df["designation_tr"].value_counts())

27633

import gmaps
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html

locations = df[['lat', 'long']]
weights = df['quality']
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap_layer)
embed_minimal_html('export.html', views=[fig])

In [ ]:
df["variety"].value_counts()[:50]

In [ ]:
df["province"].value_counts()[:50]

df["region_complete"] = df["region_complete"].replace(r'^\s*$', np.NaN, regex=True)


In [12]:
df = pd.get_dummies(df, columns=["region_complete","variety"])

In [13]:
df.to_csv("competition_dataset/mine_dev_with_dummies.csv",index=False)

In [3]:
df = pd.read_csv("competition_dataset/mine_dev_with_dummies.csv")

In [4]:
len(df.columns)

1824

In [ ]:
from platform import python_version

print(python_version())


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Preprocessing su x_eval

df_eval = pd.read_csv("competition_dataset/eval.tsv", sep="\t")


df_eval["designation_tr"] = df_eval["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df_eval["designation_tr"] = df_eval["designation_tr"].replace(["reserva","riserva","rèserve"],["reserve","reserve","reserve"])

df_eval.loc[df_eval["region_1"] == df_eval["region_2"], "region_2"] = ""
df_eval["region_complete"] = df_eval[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)
df_eval["region_complete"] = df_eval["region_complete"].replace(r'^\s*$', np.NaN, regex=True)

df_eval["region_complete"].fillna('None', inplace=True)
df_eval["variety"].fillna('None', inplace=True)
df_eval["country"].fillna('None', inplace=True)
df_eval["designation_tr"].fillna('None', inplace=True)

In [82]:
# Preprocessing su x
df = pd.read_csv("competition_dataset/dev.tsv", sep="\t")

df["designation_tr"] = df["designation"].apply(lambda d: d.lower() if pd.isna(d) == False else d)
df["designation_tr"] = df["designation_tr"].replace(["reserva","riserva","réserve"],["reserve","reserve","reserve"])

df.loc[df["region_1"] == df["region_2"], "region_2"] = ""
df["region_complete"] = df[["region_1","region_2"]].fillna(" ").agg(' '.join, axis=1)
df["region_complete"] = df["region_complete"].replace(r'^\s*$', np.NaN, regex=True)

df["region_complete"].fillna('None', inplace=True)
df["variety"].fillna('None', inplace=True)
df["country"].fillna('None', inplace=True)
df["designation_tr"].fillna('None', inplace=True)

In [18]:
df

,country,description,designation,province,region_1,region_2,variety,winery,quality,designation_tr,region_complete
0,France,"A creamed pear wine, with an attractive tang o...",Brut Blanc de Blancs,Alsace,Crémant d'Alsace,NaN,Pinot Blanc,Lucien Albrecht,45.0,brut blanc de blancs,Crémant d'Alsace
1,US,"Simple and dry, this Cabernet has modest black...",NaN,California,Paso Robles,Central Coast,Cabernet Sauvignon,Castle Rock,31.0,None,Paso Robles Central Coast
2,US,"This lovely wine captures the floral, perfumed...",NaN,Oregon,Willamette Valley,,Gewürztraminer,Château Bianca,35.0,None,Willamette Valley
3,Portugal,"The aromas are the thing here, as so often wit...",NaN,Alentejano,NaN,NaN,Touriga Nacional,Herdade do Esporão,41.0,None,None
4,Italy,"This is an interesting, outright strange wine ...",Natì,Southern Italy,Pompeiano,NaN,Coda di Volpe,Sorrentino,37.0,natì,Pompeiano
...,...,...,...,...,...,...,...,...,...,...,...
120739,Italy,This is a lovely Sauvignon with very character...,NaN,Northeastern Italy,Friuli,NaN,Sauvignon,Plozner,40.0,None,Friuli
120740,Spain,"Opens somewhat clipped and choppy, with citrus...",Reserva,Northern Spain,Rioja,NaN,Tempranillo Blend,Ramón Bilbao,41.0,reserve,Rioja
120741,Australia,Surprisingly complete and harmonious for a Rie...,NaN,South Australia,Adelaide Hills,NaN,Riesling,Wagtail,26.0,None,Adelaide Hills
120742,Germany,Broad aromas of ripe yellow fruit and a hint o...,Spätlese,Mosel,NaN,NaN,Riesling,Pamona Cellars,26.0,spätlese,None


#Aggiungo coordinate lat long a x
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df = pd.merge(df, df_region_coordinates, how="left", on="region_complete")

print(df.columns)
lat_mean = df['lat'].mean()
long_mean = df['long'].mean()

df["lat"].fillna(lat_mean, inplace=True)
df["long"].fillna(long_mean, inplace=True)


#Aggiungo coordinate lat long a x_eval
df_region_coordinates = pd.read_csv('region_coordinates.csv')

df_eval = pd.merge(df_eval, df_region_coordinates, how="left", on="region_complete")

In [83]:
# Aggiungo a x bit di encoding di region_complete e variety e country
from sklearn.preprocessing import OneHotEncoder

ohc_r = OneHotEncoder(handle_unknown="ignore")
ohc_v = OneHotEncoder(handle_unknown="ignore")
ohc_c = OneHotEncoder(handle_unknown="ignore")

ohRegion = ohc_r.fit_transform(df["region_complete"].values.reshape(-1,1)).toarray()
ohVariety = ohc_v.fit_transform(df["variety"].values.reshape(-1,1)).toarray()
ohCountry = ohc_c.fit_transform(df["country"].values.reshape(-1,1)).toarray()

dfOneHot_r = pd.DataFrame(ohRegion, columns=["region_complete_" + str(ohc_r.categories_[0][i]) for i in range(len(ohc_r.categories_[0]))])
dfOneHot_v = pd.DataFrame(ohVariety, columns=["variety_" + str(ohc_v.categories_[0][i]) for i in range(len(ohc_v.categories_[0]))])
dfOneHot_c = pd.DataFrame(ohCountry, columns=["country_" + str(ohc_c.categories_[0][i]) for i in range(len(ohc_c.categories_[0]))])

df = pd.concat([df,dfOneHot_r,dfOneHot_v, dfOneHot_c], axis=1)

In [57]:
# Aggiungo a x_eval bit di encoding di region_complete e variety

ohRegion = ohc_r.transform(df_eval["region_complete"].values.reshape(-1,1)).toarray()
ohVariety = ohc_v.transform(df_eval["variety"].values.reshape(-1,1)).toarray()
ohCountry = ohc_c.transform(df_eval["country"].values.reshape(-1,1)).toarray()

dfOneHot_r = pd.DataFrame(ohRegion, columns=["region_complete_" + str(ohc_r.categories_[0][i]) for i in range(len(ohc_r.categories_[0]))])
dfOneHot_v = pd.DataFrame(ohVariety, columns=["variety_" + str(ohc_v.categories_[0][i]) for i in range(len(ohc_v.categories_[0]))])
dfOneHot_c = pd.DataFrame(ohCountry, columns=["country_" + str(ohc_c.categories_[0][i]) for i in range(len(ohc_c.categories_[0]))])

df_eval = pd.concat([df_eval,dfOneHot_r,dfOneHot_v,dfOneHot_c], axis=1)

print(df.shape, df_eval.shape)

(120744, 1874) (30186, 1873)


["stephanie's vineyard",
 "terry vintner's family reserve",
 'mon coeur',
 "d'antan",
 'mark v',
 'westhofener kirchspiel auslese s',
 'blanc de gris dry white',
 'zédé de labégorce',
 'vandal vineyard',
 'les pierres franches',
 'pierrefolle',
 'francesca',
 'cuvée c.m.',
 'piesporter goldtröpfchen spätlese grosse lage',
 "vintner's selection white horse red",
 'terra rosa',
 'albert',
 'cinnamon hill vineyard la morra',
 'eileen',
 'library vineyard',
 'cabyrah',
 'naturalmente bio',
 'el buho red wine',
 'cellar no. 9',
 'gradum reserve',
 'saarburger old vines trocken',
 'amado sur chardonnay-pinot grigio-viognier',
 "d'loiodice zinfandel rosá",
 'r m nicholson',
 'lot 466',
 'rough justice',
 'spécial club grand cru brut',
 '30 yr. old baco imperial oloroso',
 'blonde de seigneur',
 "huckleberry's vineyard",
 'fastaia',
 'crucero collection',
 'voulez-vous',
 'assobio',
 "l'esprit de marius sangouard",
 'lolol',
 'abstberg beerenauslese',
 'm. coz',
 'the aviator',
 'iv',
 'reserv

In [84]:
#Aggiungo i bit dei primi N designation a x
from sklearn.preprocessing import OneHotEncoder

N = 300
top_N_designation_tr = list(df["designation_tr"].value_counts()[1:N+1].keys()) #non includo None values

df.loc[~df["designation_tr"].isin(top_N_designation_tr) ,"designation_tr"] = "None"

ohc_d = OneHotEncoder(handle_unknown="ignore")

ohDesignation = ohc_d.fit_transform(df["designation_tr"].values.reshape(-1,1)).toarray()
dfOneHot_d = pd.DataFrame(ohDesignation, columns=["designation_tr_" + str(ohc_d.categories_[0][i]) for i in range(len(ohc_d.categories_[0]))])

df = pd.concat([df,dfOneHot_d], axis=1)

In [59]:
#Aggiungo i bit dei primi N designation a x eval
from sklearn.preprocessing import OneHotEncoder

N = 300
ohDesignation = ohc_d.transform(df_eval["designation_tr"].values.reshape(-1,1)).toarray()
dfOneHot_d = pd.DataFrame(ohDesignation, columns=["designation_tr_" + str(ohc_d.categories_[0][i]) for i in range(len(ohc_d.categories_[0]))])

df_eval = pd.concat([df_eval,dfOneHot_d], axis=1)

In [85]:
#Creo Vectorizer per x e x_eval
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english")

wpm = vectorizer.fit_transform(df["description"].fillna(""))

In [86]:
#Aggiungo a x i bit delle N parole più frequenti nella description 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", binary=True, use_idf=False,norm=False)

wpm = vectorizer.fit_transform(df["description"].fillna(""))

N = 500
freq = sorted(zip(vectorizer.get_feature_names(), wpm.sum(axis=0).tolist()[0]),key=lambda x: x[1], reverse=True)[:N]

words = [ word for word, _ in freq ]
mask = [ w in words for w in vectorizer.get_feature_names() ]
words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
df_words = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df.index)

df = pd.concat([df,df_words], axis=1)

In [62]:
#Aggiungo a x_eval i bit delle N parole più frequenti nella description 
from sklearn.feature_extraction.text import TfidfVectorizer

wpm = vectorizer.transform(df_eval["description"].fillna(""))


df_words_eval = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df_eval.index)

df_eval = pd.concat([df_eval,df_words_eval], axis=1)


#Test tdidf description su x
from sklearn.feature_extraction.text import TfidfVectorizer

N = 500
vectorizer = TfidfVectorizer(stop_words="english", use_idf=True, max_features = N)

wpm = vectorizer.fit_transform(df["description"].fillna(""))

words = [ word for word, _ in freq ]
mask = [ w in words for w in vectorizer.get_feature_names() ]
words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
df_words = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(),columns=[f"word_{word}" for word in words_], index=df.index)

df_words

,word_10,word_100,word_15,word_20,word_2010,word_2012,word_2015,word_2016,word_2017,word_2018,...,word_wood,word_wrapped,word_year,word_years,word_yellow,word_young,word_zest,word_zesty,word_zin,word_zinfandel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
30183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor(n_estimators=100, n_jobs=-1,max_features="sqrt")

In [13]:
df.iloc[:,9:]

,designation_tr,region_complete,region_complete_Abruzzo,region_complete_Adelaida District Central Coast,region_complete_Adelaide,region_complete_Adelaide Hills,region_complete_Adelaide Plains,region_complete_Aglianico del Beneventano,region_complete_Aglianico del Taburno,region_complete_Aglianico del Vulture,...,word_wood,word_wrapped,word_year,word_years,word_yellow,word_young,word_zest,word_zesty,word_zin,word_zinfandel
0,None,Crémant d'Alsace,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,None,Paso Robles Central Coast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,None,Willamette Valley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,None,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,None,Pompeiano,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120739,None,Friuli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120740,reserve,Rioja,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120741,None,Adelaide Hills,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120742,spätlese,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [74]:
#Test PCA 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = df.iloc[:, 11:]
y = df["quality"]

scaler = MinMaxScaler()
X_rescaled = scaler.fit_transform(X)
#y_rescaled = scaler.fit_transform(y)

pca = PCA(n_components = 0.99)
pca.fit(X_rescaled)
reduced = pca.transform(X_rescaled)

reduced


array([[-4.07232191e-01, -1.18040601e+00, -6.78016697e-01, ...,
         1.05854083e-02,  1.34933279e-03,  5.49092371e-03],
       [ 1.29398145e+00,  3.36636898e-01,  4.66573099e-01, ...,
         1.23762805e-03, -4.85587040e-06, -2.15115329e-04],
       [ 3.24890915e-01, -6.82529368e-01, -2.73128777e-01, ...,
        -4.89591051e-04,  2.82915083e-04, -5.28380172e-03],
       ...,
       [ 1.31597838e-02,  4.35931765e-01, -8.77777397e-01, ...,
        -3.27338856e-03, -2.77930126e-03, -2.81970289e-03],
       [-7.35400636e-01,  9.36866593e-01, -7.09643064e-01, ...,
        -1.99616790e-02,  6.26843273e-03,  1.27249550e-02],
       [-6.07383196e-01, -9.62114352e-01, -1.99063578e-01, ...,
        -1.85599034e-03,  2.00852126e-03,  1.91786420e-04]])

In [76]:
reduced.shape

(120744, 966)

In [75]:
# Predict con PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
            
X_train, X_test, y_train, y_test = train_test_split(reduced, y, test_size=0.25, random_state=34)

regr.fit(X_train,y_train)
            
y_pred = regr.predict(X_test)

r2_score(y_test, y_pred)

0.6441551847443059

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X = df.iloc[:, 11:]
y = df["quality"]
            
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=34)

regr.fit(X_train,y_train)
            
y_pred = regr.predict(X_test)

r2_score(y_test, y_pred)

0.696325906426519

In [64]:
#Ricerca con grid search

from sklearn.model_selection import GridSearchCV

X = df.iloc[:, 11:]
y = df["quality"]
            
param_grid = {
"n_estimators": [100, 250],
"criterion": ["mse"],
"max_features": ["auto", "sqrt", "log2"],
"random_state": [42], # always use the samet random seed
"n_jobs": [-1], # for parallelization
}

gs = GridSearchCV(RandomForestRegressor(), param_grid, scoring="r2", n_jobs=-1,cv=5)
gs.fit(X, y)
gs.best_score_

KeyboardInterrupt: 

In [ ]:
from prettytable import PrettyTable

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

def evaluate_model(X, y, model, model_name):

    X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.25,random_state=42,shuffle=True)
    # plot the real function and the training points
    LW = 2
    #fig, ax = plt.subplots()
    #ax.plot(X, y, color='cornflowerblue', linewidth=.5*LW, label="ground truth")
    #ax.scatter(X_train, y_train, color='navy', s=30, marker='o',label="training points")
    # predict the test points and plot them onto the chart
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    #ax.plot(X_test, y_hat, linewidth=LW, label=name, color='r')
    #fig.suptitle(f"{f} approximated by {model_name}")
    #fig.legend()
    return r2_score(y_test, y_hat)



degree = 2
models = [
    LinearRegression(),
    Ridge(random_state=42),
    SVR(gamma='scale'),
    RandomForestRegressor(n_estimators=100,n_jobs=-1,max_features="sqrt"),
    make_pipeline(
        make_column_transformer(
            (FunctionTransformer(np.sin), [0]),
            (PolynomialFeatures(degree), [0])
        ),
        LinearRegression()
    ),
    make_pipeline(
        make_column_transformer(
            (FunctionTransformer(np.sin), [0]),
            (PolynomialFeatures(degree), [0])
        ),
        Ridge(alpha=1)
    )
]

names = [
    'linreg',
    'ridge',
    'svr',
    'rf',
    f'sin+poly{degree}+linreg',
    f'sin+poly{degree}+ridge'
]


t = PrettyTable()
t.field_names = ['model', 'R2']
X = df.iloc[:, 11:]
y = df["quality"]
for model, name in zip(models, names):
    print(model, name)
    r2 = evaluate_model(X, y, model, name)
    t.add_row([name, r2])
print(t)

In [11]:
freq[:100]

[('wine', 60287.0),
 ('flavors', 59577.0),
 ('fruit', 43327.0),
 ('finish', 29935.0),
 ('aromas', 28425.0),
 ('acidity', 25570.0),
 ('cherry', 25048.0),
 ('tannins', 25011.0),
 ('palate', 23127.0),
 ('ripe', 20615.0),
 ('dry', 19235.0),
 ('drink', 19152.0),
 ('black', 18783.0),
 ('spice', 17699.0),
 ('sweet', 17024.0),
 ('rich', 16516.0),
 ('oak', 15876.0),
 ('red', 15024.0),
 ('notes', 14916.0),
 ('soft', 13847.0),
 ('berry', 13822.0),
 ('fresh', 13702.0),
 ('good', 13044.0),
 ('nose', 12742.0),
 ('shows', 12329.0),
 ('blackberry', 11889.0),
 ('crisp', 10992.0),
 ('vanilla', 10820.0),
 ('blend', 10760.0),
 ('plum', 10485.0),
 ('years', 9976.0),
 ('chocolate', 9691.0),
 ('citrus', 9514.0),
 ('fruits', 9446.0),
 ('dark', 9438.0),
 ('mouth', 9411.0),
 ('white', 8907.0),
 ('like', 8856.0),
 ('apple', 8724.0),
 ('light', 8622.0),
 ('cabernet', 8588.0),
 ('texture', 8523.0),
 ('offers', 8447.0),
 ('green', 8373.0),
 ('raspberry', 7998.0),
 ('bright', 7959.0),
 ('pepper', 7374.0),
 ('balance

In [43]:
X.shape

(120744, 2164)

['region_complete_Albana di Romagna  ',
 'region_complete_Albemarle County  ',
 'region_complete_Alcamo  ',
 "region_complete_Aleatico dell'Elba  ",
 'region_complete_Alella  ',
 'region_complete_Alexander Valley Sonoma',
 'region_complete_Alghero  ',
 'region_complete_Alicante  ',
 'region_complete_Alicante-Marina Alta  ',
 'region_complete_Almansa  ',
 'region_complete_Aloxe-Corton  ',
 'region_complete_Alpilles  ',
 'region_complete_Alsace  ',
 'region_complete_Alta Langa  ',
 'region_complete_Alta Mesa Central Valley',
 'region_complete_Alta Valle della Greve  ',
 'region_complete_Alto Adige  ',
 'region_complete_Alto Adige Terlano  ',
 'region_complete_Alto Adige Valle Isarco  ',
 'region_complete_Alto Valle de Uco  ',
 'region_complete_Alto Valle del Río Negro  ',
 'region_complete_Amador County Sierra Foothills',
 'region_complete_Amador-Napa California Other',
 'region_complete_Amarone della Valpolicella  ',
 'region_complete_Amarone della Valpolicella Classico  ',
 'region_com

In [90]:
sorted(zip(list(X.columns[11:]), regr.feature_importances_), key=lambda x: x[1],reverse=True)

[('region_complete_Capay Valley Central Valley', 0.018476182275526765),
 ('variety_Portuguese White', 0.01551639288447137),
 ('word_toasted', 0.009679635788763125),
 ('country_Hungary', 0.009214853755370025),
 ('region_complete_Offida Passerina  ', 0.008728601776022587),
 ('word_rustic', 0.008184738429986899),
 ('word_ready', 0.007546732201514818),
 ('country_Czech Republic', 0.007263385094193819),
 ('word_2016', 0.007258273554653265),
 ('region_complete_Chianti  ', 0.007232243115176982),
 ('word_approachable', 0.006229087284883107),
 ('region_complete_Napa-Sonoma-Monterey North Coast', 0.006157010035524896),
 ('word_zesty', 0.005974264003779842),
 ('word_winemaker', 0.005898405004409457),
 ('country_Luxembourg', 0.005849337448105694),
 ('word_bone', 0.005745960058491417),
 ('word_french', 0.005687122323449621),
 ('region_complete_Calabria  ', 0.005683346045060075),
 ('word_edge', 0.0056723942743756505),
 ('variety_Nuragus', 0.0055079415551769996),
 ('word_complex', 0.00536355884729412

In [125]:
X_train["lat"].isnull().sum()


14094

In [107]:
df_eval.iloc[:,9:]

,region_complete,region_complete_Abruzzo,region_complete_Adelaida District Central Coast,region_complete_Adelaide,region_complete_Adelaide Hills,region_complete_Adelaide Plains,region_complete_Aglianico del Beneventano,region_complete_Aglianico del Taburno,region_complete_Aglianico del Vulture,region_complete_Agrelo,...,word_wood,word_wrapped,word_year,word_years,word_yellow,word_young,word_zest,word_zesty,word_zin,word_zinfandel
0,Santa Cruz Mountains Central Coast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Toro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Saint-Julien,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Cahors,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30181,Napa Valley Napa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30182,Dry Creek Valley Sonoma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
30183,Santa Barbara County Central Coast,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30184,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, 11:]
y = df["quality"]
            
regr.fit(X,y)

RandomForestRegressor(max_features='sqrt', n_jobs=-1)

In [109]:
X_eval = df_eval.iloc[:,10:]

y_eval = regr.predict(X_eval)

In [94]:
y_eval

array([55.11174904, 47.33763215, 49.09385491, ..., 48.99112121,
       28.23503571, 38.79995463])

In [110]:
with open("my_submission.csv","w") as f:
    f.write("Id,Predicted\n")
    for i,result in enumerate(y_eval):
        f.write(str(i) + "," + str(y_eval[i])+"\n")